In [1]:
import pandas as pd
import re

In [2]:
!git clone https://github.com/DV-11/SpanishDialectDiscrimination.git

Cloning into 'SpanishDialectDiscrimination'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 3), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 10.98 MiB | 5.50 MiB/s, done.
Resolving deltas: 100% (3/3), done.


# Loading the data

In [3]:
preseea_df = pd.read_csv('/content/SpanishDialectDiscrimination/Data/PRESEEA.csv')

In [4]:
preseea_es = preseea_df[preseea_df['country'] == 'España'].reset_index()['text']
preseea_mx = preseea_df[preseea_df['country'] == 'México'].reset_index()['text']

In [5]:
print("Numer of PS dialogues: ", len(preseea_es))
print("Numer of MS dialogues: ", len(preseea_mx))

Numer of PS dialogues:  233
Numer of MS dialogues:  90


Find specific entry index by text ID

In [6]:
text_id = 'MADR_H11_002'
dataset = preseea_es

dataset.str.contains(text_id).idxmax()

36

# Parsing dialogue

In [7]:
# es - 37 ; mx - 19
dialogue = preseea_mx[19]


In [8]:
split_str = dialogue.split('</Trans>')
info = split_str[0]
transcript = split_str[1]

In [9]:
all = re.findall(r'(E:|I:)(.*?)(?=(?:E:|I:)|$)', transcript)

dialogue_lines = []
current_line = {'E_line': None, 'I_line': None}

for i, j in all:
    stripped_line = j.strip().removesuffix('/').strip()

    if i == 'E:':
        if current_line['I_line'] is not None:
            dialogue_lines.append(current_line)
            current_line = {'E_line': None, 'I_line': None}
        current_line['E_line'] = stripped_line
    elif i == 'I:':
        current_line['I_line'] = stripped_line

    if current_line['E_line'] is not None and current_line['I_line'] is not None:
        dialogue_lines.append(current_line)
        current_line = {'E_line': None, 'I_line': None}

if current_line['E_line'] is not None or current_line['I_line'] is not None:
    dialogue_lines.append(current_line)

In [10]:
dialogue_df = pd.DataFrame(dialogue_lines)
dialogue_df['Line_number'] = dialogue_df.index + 1
dialogue_df = dialogue_df[['Line_number', 'E_line', 'I_line']]

dialogue_df.head()

,Line_number,E_line,I_line
0,1,¿cómo empezaste a bailar?,"<tiempo = ""00:02""/> ¿cómo empecé a bailar? // ..."
1,2,¿y?,y pues es / <simultáneo> fue eso </simultáneo>
2,3,<simultáneo> pero porque </simultáneo> es exte...,porque es externa <simultáneo> porque </simult...
3,4,<simultáneo> ¿pero es </simultáneo> externa na...,sí / porque / y ya / bueno ya / de ahí / no me...
4,5,mh,y ya de ahí pues / conocí la danza / en lo que...


Clean up unnecessary tags

In [11]:
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('<alargamiento/>','')
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('<vacilación/>','')
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('</simultáneo>','')
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('<simultáneo>','')

In [12]:
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('<alargamiento/>','')
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('<vacilación/>','')
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('</simultáneo>','')
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('<simultáneo>','')

# Dialogue Info

In [13]:
tags = [i for i in info.split(' ') if '=' in i]

all_info = {}

for i in tags:
  split_tag = i.split('=')
  k = split_tag[0]
  v = split_tag[1]

  if not re.findall('"([^"]*)"',split_tag[1]):
    all_info[split_tag[0]] = None
  else:
    all_info[split_tag[0]] = re.findall('"([^"]*)"',split_tag[1])[0]

important_keys = ['audio_filename','clave_texto', 'pais', 'fecha_grab', 'fecha_trans']

dialogue_info = {key: all_info[key] for key in important_keys}

In [14]:
info

'<Trans audio_filename="MEXI_H12_042.wav" xml:lang="español"> <Datos clave_texto="MEXI_H12_042" tipo_texto="entrevista_semidirigida"> <Corpus corpus="PRESEEA" subcorpus="PRESEEA_MÉXICO" ciudad="México, D. F." pais="México"/>  <Grabacion resp_grab="Alejandra Aide Espinoza Vázquez" lugar="Escuela Nacional de Danza Folclórica" duración="90\'" fecha_grab="2006-04-17" sistema="DAT"/> <Transcripcion resp_trans="J. Rodríguez" fecha_trans="2006-07-07" numero_palabras="15541"/> <Revision num_rev="1" resp_rev="A. Salas" fecha_rev="2006-12-11"/> <Revision num_rev="2" resp_rev="T. García Torres" fecha_rev="2007-02-28"/> <Revision num_rev="3" resp_rev="L. Roldán" fecha_rev="2007-04-03"/> <Revision num_rev="4" resp_rev="P. Martín" fecha_rev="2010-07-31"/> <Revision num_rev="5" resp_rev="P. Ledesma" fecha_rev="2017-05-05"/></Datos>  <Hablantes> <Hablante id="hab1" nombre=" MEXI_H12_042" código_hab="I" sexo="hombre" Grupo_edad="1" edad="20" nivel_edu="medio" estudios="bachillerato" profesión="estudian

In [15]:
dialogue_info

{'audio_filename': 'MEXI_H12_042.wav',
 'clave_texto': 'MEXI_H12_042',
 'pais': 'México',
 'fecha_grab': '2006-04-17',
 'fecha_trans': '2006-07-07'}

# Readable transcript

In [16]:
for i in dialogue_df.iterrows():
  print(i[1].Line_number, 'E:', i[1].E_line)
  print(i[1].Line_number, 'I:', i[1].I_line)
  print('')

1 E: ¿cómo empezaste a bailar?
1 I: <tiempo = "00:02"/> ¿cómo empecé a bailar? // fue bien / bueno / fue chistoso porque este / empecé con un profesor / que montaba puros quince años // y ya de ahí me invitó a salir en quince años / y ya ves / ahí vas de metiche / pero para esto // yo lo conocí a él porque  en la secundaria / una de mis mejores amigas / me pidió que fuera su chambelán / le dije que sí / entonces ya ahí conocí a él / y de ahí / de ahí este / se llama J G // me empezó a jalar a / a montar quince años / y me pagaba y ya yo estaba con él / y de repente <ininteligible/> él / él baila folclor y se le ocurrió formar su grupo folclórico // entonces yo fui uno de los que empezó con su ballet de él / y yo / ahí / netamente el folclor / ahí lo conocí // y ya de ahí em <palabra_cortada/> / pues me empezó a gustar mucho y / y le empecé a echar ganas y em <palabra_cortada/> / empecé a bailar / un poquito mejor / y ya de repente / pues / él me comentó de la escuela / entonces ya este